# Mouse: M044
## Session: 2024_12_04_09_30

#### Imports

In [24]:
# imports
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add the project root to sys.path
import sys
sys.path.append('/data/PyalData')

import pyaldata as pyal
import pylab
from collections import Counter
import importlib

from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# for RNN and CURBD
from tools.dsp.preprocessing import preprocess
from tools.rnn_and_curbd import curbd_and_rnn
from tools.rnn_and_curbd.curbd_and_rnn import *
importlib.reload(curbd_and_rnn)

np.random.seed(44)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load session data

In [17]:
data_dir = "/data/bnd-data/raw/M044/M044_2024_12_04_09_30"
mat_file = "M044_2024_12_04_09_30_pyaldata.mat"

In [19]:
fname = os.path.join(data_dir, mat_file)
df = pyal.mat2dataframe(fname, shift_idx_fields=True)

field values_before_camera_trigger could not be converted to int.
field idx_before_camera_trigger could not be converted to int.
array field all_KSLabel could not be converted to int.


### Preprocessing

In [20]:
df_ = preprocess(df, only_trials=False)
areas = ["M1", "Dls"]
df_["M1_rates"] = [df_["all_rates"][i][:,300:] for i in range(len(df_))]
df_["Dls_rates"] = [df_["all_rates"][i][:,0:300] for i in range(len(df_))]

/data/PyalData/pyaldata/firing_rates.py:108: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn(


Combined every 3 bins
Resulting all_spikes ephys data shape is (NxT): (474, 20000)


In [21]:
df_.head()

,animal,session,trial_id,trial_name,trial_length,bin_size,idx_trial_start,idx_trial_end,idx_CPI,values_before_camera_trigger,...,idx_session_timer,all_chan_best,all_unit_guide,all_KSLabel,all_spikes,all_rates,sol_level_id,sol_contra_ipsi,M1_rates,Dls_rates
0,M044,M044_2024_12_04_09_30,0,free,60001,0.03,-1,19999,[],1.0,...,[],"[0, 0, 0, 1, 1, 1, 2, 3, 3, 3, 3, 3, 5, 5, 5, ...","[[0, 1], [5, 1], [5, 3], [6, 2], [8, 2], [9, 1...","[good, mua, good, mua, good, good, good, good,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[12.95344928900401, 0.0, 0.0, 10.218119621276...",NaN,NaN,"[[0.9141243373354276, 0.0, 0.2123920402179162,...","[[12.95344928900401, 0.0, 0.0, 10.218119621276..."
1,M044,M044_2024_12_04_09_30,1,intertrial,101,0.03,19999,20033,[],[],...,[],"[0, 0, 0, 1, 1, 1, 2, 3, 3, 3, 3, 3, 5, 5, 5, ...","[[0, 1], [5, 1], [5, 3], [6, 2], [8, 2], [9, 1...","[good, mua, good, mua, good, good, good, good,...","[[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,...","[[9.000157316525778, 0.0, 0.0, 0.0, 12.9494756...",NaN,NaN,"[[0.0, 0.04343213513493538, 0.0, 0.0, 0.0, 20....","[[9.000157316525778, 0.0, 0.0, 0.0, 12.9494756..."
2,M044,M044_2024_12_04_09_30,2,trial,401,0.03,20033,20166,[],[],...,[],"[0, 0, 0, 1, 1, 1, 2, 3, 3, 3, 3, 3, 5, 5, 5, ...","[[0, 1], [5, 1], [5, 3], [6, 2], [8, 2], [9, 1...","[good, mua, good, mua, good, good, good, good,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 2.7987738450368247, 0.0, 2.79...",0.0,0.0,"[[0.0003196751701276905, 0.00397311378037557, ...","[[0.0, 0.0, 0.0, 2.7987738450368247, 0.0, 2.79..."
3,M044,M044_2024_12_04_09_30,3,intertrial,101,0.03,20166,20199,[],[],...,[],"[0, 0, 0, 1, 1, 1, 2, 3, 3, 3, 3, 3, 5, 5, 5, ...","[[0, 1], [5, 1], [5, 3], [6, 2], [8, 2], [9, 1...","[good, mua, good, mua, good, good, good, good,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[10.218119621276855, 0.0, 0.0, 0.000319675170...",NaN,NaN,"[[0.03477109130471945, 0.0, 0.0, 0.0, 0.0, 24....","[[10.218119621276855, 0.0, 0.0, 0.000319675170..."
4,M044,M044_2024_12_04_09_30,4,trial,401,0.03,20199,20333,[],[],...,[],"[0, 0, 0, 1, 1, 1, 2, 3, 3, 3, 3, 3, 5, 5, 5, ...","[[0, 1], [5, 1], [5, 3], [6, 2], [8, 2], [9, 1...","[good, mua, good, mua, good, good, good, good,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.9180974215269089,...",0.0,1.0,"[[0.00397311378037557, 0.0, 12.949475646018982...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.9180974215269089,..."


### Some parameters

In [26]:
# Get mouse number
mouse = mat_file.split('_')[0]

# perturbation time
perturbation_time = df_.idx_sol_on[0]
perturbation_time_sec = df_.idx_sol_on[0] * df['bin_size'][0]

# solenoid angles
sol_angles: list = df_.values_Sol_direction.unique()
sol_angles.sort()

# number of trials
num_trials = len(df_)
print(f"Number of trials: {num_trials}")

# Set interpolation factor for RNN 
dtFactor = 2

TypeError: unhashable type: 'numpy.ndarray'

### Conactanate trials only

In [22]:
# getting trial avg data
trial_avg_rates = average_by_trial(df_, sol_angles)
shapes = [arr.shape[0] for arr in trial_avg_rates]

# concatenate data
concat_rates = np.concatenate(trial_avg_rates, axis=0)

print("Averaged activity shape:", trial_avg_rates.shape)
print("Concatenated average activity shape:", concat_rates.shape)

# set up for RNN
formated_rates = np.transpose(concat_rates)
resets = get_reset_points(df_, formated_rates, areas, dtFactor)
regions_arr = get_regions(df_, areas)

print(f"Building {len(regions_arr)} region RNN network")
print(f"Regions: {[region[0] for region in regions_arr]}\n")

NameError: name 'average_by_trial' is not defined